# Real example of ML



## Libraries

In [ ]:
# ============================================
# SEZIONE: Importazione Librerie
# ============================================
# In questa sezione importiamo tutte le librerie necessarie per il progetto di Machine Learning

# --- Librerie Base per Data Science ---
import numpy as np          # Libreria per calcoli numerici e operazioni su array
import pandas as pd         # Libreria per manipolare dati in formato tabellare (DataFrame)
import matplotlib.pyplot as plt  # Libreria per creare grafici e visualizzazioni
import seaborn as sns       # Libreria per visualizzazioni statistiche avanzate

# --- Strumenti per la preparazione dei dati e validazione ---
from sklearn.model_selection import train_test_split, KFold, cross_validate
# train_test_split: divide i dati in training e test set
# KFold: divide i dati in k "pieghe" per la cross-validazione
# cross_validate: esegue la validazione incrociata

# --- Metriche di valutazione ---
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, make_scorer
# mean_squared_error (MSE): errore quadratico medio
# root_mean_squared_error (RMSE): radice dell'errore quadratico medio
# mean_absolute_error (MAE): errore assoluto medio
# make_scorer: crea metriche personalizzate

# --- Tecniche di scaling/normalizzazione ---
from sklearn.preprocessing import StandardScaler   # Standardizza i dati (media=0, deviazione standard=1)
from sklearn.preprocessing import MinMaxScaler     # Normalizza i dati tra 0 e 1
from sklearn.preprocessing import RobustScaler     # Scala i dati riducendo l'effetto degli outlier

# --- Pipeline per concatenare trasformazioni ---
from sklearn.pipeline import Pipeline  # Permette di creare flussi di lavoro concatenati

# --- Modelli di Machine Learning ---
from sklearn.dummy import DummyRegressor           # Modello baseline che fa predizioni semplici
from sklearn.linear_model import LinearRegression  # Regressione lineare classica
from sklearn.neighbors import KNeighborsRegressor  # Regressione basata sui k vicini più prossimi
from sklearn.tree import DecisionTreeRegressor, plot_tree  # Albero decisionale per regressione

# --- Strumenti per ottimizzazione iperparametri ---
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# GridSearchCV: ricerca esaustiva dei migliori iperparametri
# RandomizedSearchCV: ricerca casuale dei migliori iperparametri (più veloce)

## Load Data

In [ ]:
# ============================================
# SEZIONE: Caricamento dei Dati
# ============================================
# Carichiamo il dataset delle case dal file CSV
# CSV = Comma Separated Values (file con valori separati da virgole)

try:
    # Proviamo a leggere il file CSV con i dati delle case
    df = pd.read_csv('gb_house_pricing.csv')
    
    # Se il caricamento ha successo, mostriamo un messaggio di conferma
    print(f"✓ File caricato con successo: {len(df)} righe, {len(df.columns)} colonne")
    
except FileNotFoundError:
    # Se il file non viene trovato, mostriamo un errore chiaro
    print("❌ Errore: file 'gb_house_pricing.csv' non trovato!")
    print("   Assicurati che il file sia nella stessa cartella del notebook")
    raise  # Rilancia l'errore per fermare l'esecuzione
    
except Exception as e:
    # Se ci sono altri errori durante il caricamento
    print(f"❌ Errore nel caricamento del file: {e}")
    raise  # Rilancia l'errore per fermare l'esecuzione

# Impostiamo l'opzione per visualizzare tutte le colonne quando stampiamo il DataFrame
# Normalmente pandas nasconde alcune colonne se sono troppe
pd.set_option('display.max_columns', None)

# Mostriamo le prime 5 righe del dataset per avere un'idea dei dati
# head() = "testa" in inglese, cioè mostra la parte iniziale dei dati
df.head()

In [3]:
# Visualizziamo le dimensioni del dataset
# shape restituisce una tupla (numero_righe, numero_colonne)
# Questo ci aiuta a capire quanti dati abbiamo a disposizione
df.shape


(992, 37)

Suddivisione fadi specific per questo caso. 

In [ ]:
# ============================================
# SEZIONE: Separazione dei Dati
# ============================================
# Il dataset contiene una colonna 'Split' che indica se i dati sono:
# - 'labeled': dati con etichetta (SalePrice), usati per training/test
# - 'leaderboard': dati senza etichetta, da usare per predizioni finali

# Creiamo un DataFrame separato per i dati di training/test
# .copy() crea una copia indipendente per evitare problemi di modifica
train_df = df[df['Split'] == 'labeled'].copy()

# Creiamo un DataFrame separato per i dati della leaderboard (predizioni finali)
leaderboard_test_df = df[df['Split'] == 'leaderboard'].copy()

# NOTA: la variabile 'last_pred' verrà creata più avanti nel notebook
# quando faremo le predizioni finali sul set di leaderboard

In [5]:
# Visualizziamo le dimensioni del set di training
# Questo ci mostra quante case abbiamo per addestrare il modello
train_df.shape


(794, 37)

## Train / Test 


ora he il momento di di isolare la colonna target e omettere colonna id e split

In [6]:
# ============================================
# Separazione Features e Target
# ============================================
# Dividiamo i dati in:
# - X: features (caratteristiche) - le informazioni che il modello usa per imparare
# - y: target (obiettivo) - il valore che vogliamo predire (SalePrice = prezzo di vendita)

# Creiamo X rimuovendo le colonne non utili per la predizione:
# - 'Id': solo un identificatore
# - 'Split': indica train/test, non è una caratteristica della casa
# - 'SalePrice': è il target, non può essere una feature
X = train_df.drop(['Id', 'Split', 'SalePrice'], axis=1)

# Creiamo y prendendo solo la colonna del prezzo (quello che vogliamo predire)
y = train_df['SalePrice']

# Dividiamo i dati in training set (80%) e test set (20%)
# - Training set: usato per addestrare il modello
# - Test set: usato per valutare le prestazioni su dati mai visti
# random_state=42 garantisce che la divisione sia sempre la stessa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Visualizziamo le dimensioni dei set di training e test
# Questo ci conferma che la divisione è avvenuta correttamente
print("Dimensioni di X_train:", X_train.shape)
print("Dimensioni di X_test :", X_test.shape)
print("Dimensioni di y_train:", y_train.shape)
print("Dimensioni di y_test :", y_test.shape)


## Cross-validation

In [7]:
# ============================================
# Configurazione della Cross-Validation
# ============================================
# La cross-validation (validazione incrociata) divide i dati in K "pieghe" (folds)
# e addestra il modello K volte, ogni volta usando una piega diversa per il test

# KFold con 10 pieghe significa:
# - Dividiamo i dati in 10 parti
# - Usiamo 9 parti per addestrare e 1 per testare
# - Ripetiamo 10 volte, cambiando ogni volta la parte usata per il test
# shuffle=True: mescola i dati prima di dividerli
# random_state=42: garantisce risultati riproducibili
kf = KFold(n_splits=10, shuffle=True, random_state=42)


KFold(n_splits=10, random_state=42, shuffle=True)


In [8]:
# ============================================
# Definizione delle Metriche di Valutazione
# ============================================
# Definiamo le metriche per valutare quanto bene il nostro modello fa le predizioni:

# 1. MSE (Mean Squared Error): errore quadratico medio
#    - Più sensibile agli outlier (errori grandi pesano molto)
#    - Risultato al quadrato, quindi difficile da interpretare
# 2. RMSE (Root Mean Squared Error): radice dell'errore quadratico medio
#    - Stesso scale del target (prezzo), più facile da interpretare
# 3. MAE (Mean Absolute Error): errore assoluto medio
#    - Meno sensibile agli outlier
#    - Facile da interpretare: errore medio in dollari/euro

# Nota: sklearn usa punteggi negativi (più alto = migliore)
# quindi usiamo make_scorer con greater_is_better=False
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'RMSE': make_scorer(root_mean_squared_error, greater_is_better=False),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False)
}


{'MSE': make_scorer(mean_squared_error, response_method='predict'), 'MAE': make_scorer(mean_absolute_error, response_method='predict'), 'RMSE': make_scorer(root_mean_squared_error, response_method='predict')}


dict_keys(['MSE', 'MAE', 'RMSE'])

## Baseline

In [9]:
# ============================================
# Modello Baseline (DummyRegressor)
# ============================================
# Prima di provare modelli complessi, creiamo un modello "stupido" (baseline)
# che fa predizioni molto semplici. Questo ci dà un punto di riferimento:
# se i nostri modelli complessi non battono il baseline, c'è qualcosa che non va!

# DummyRegressor con strategy='mean' predice sempre la media del training set
# È il modello più semplice possibile
bl = DummyRegressor(strategy='mean')

# Eseguiamo la cross-validation per valutare il baseline
# cv_results conterrà i punteggi per MSE, RMSE e MAE
cv_results = cross_validate(bl, X_train, y_train, cv=kf, scoring=scoring)

# Convertiamo i risultati in un DataFrame per visualizzarli meglio
cv_results_df = pd.DataFrame(cv_results)

# Mostriamo i risultati (i valori sono negativi, ma più alti = meglio)
cv_results_df


Train MSE: 2787252534.7
Validation MSE: 2797448049.9

Train MAE: 39442.3
Validation MAE: 39501.4


## Linear Regression 

e` buona cosa sempre iniziare con un rl cosi da vedere se e´o non e` questa tipologia di dati

In [13]:
# ============================================
# Regressione Lineare con Scaling
# ============================================
# La regressione lineare cerca di trovare la migliore linea (o iperpiano)
# che passa attraverso i dati per fare predizioni

# Usiamo una Pipeline che concatena due passaggi:
# 1. StandardScaler: normalizza i dati (media=0, deviazione standard=1)
#    - Importante perché le features hanno scale diverse
# 2. LinearRegression: il modello di regressione lineare vero e proprio

lr = Pipeline([
    ('scale', StandardScaler()),     # Step 1: standardizza i dati
    ('lr', LinearRegression())       # Step 2: applica regressione lineare
])

# Eseguiamo cross-validation sulla regressione lineare
cv_results = cross_validate(lr, X_train, y_train, cv=kf, scoring=scoring)

# Convertiamo i risultati in DataFrame
cv_results_df = pd.DataFrame(cv_results)

# Mostriamo i risultati
cv_results_df


Train MSE: 286231258.2
Validation MSE: 341104355.3

Train MAE: 12413.3
Validation MAE: 13354.8


# Hyperparameter Tuning

 Find good hyperparameters for the tree-based methods and kNN.
 • Use 
 RandomizedSearchCV  and 
 GridSearchCV  from sklearn

### Hyperparameters for the tree-based RandomizedSearchCV

| **Iperparametro**          | **Tipo / Range consigliato per Randomized Search**                | **Descrizione**                                                                                                                                                                                                                                           |
| -------------------------- | ----------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `criterion`                | `["squared_error", "friedman_mse", "absolute_error", "poisson"]`  | Funzione per misurare la qualità di una suddivisione. <br>• `squared_error`: minimizza MSE (default) <br>• `friedman_mse`: ottimizzato per boosting <br>• `absolute_error`: più robusto ai valori anomali <br>• `poisson`: per target positivi (conteggi) |
| `splitter`                 | `["best", "random"]`                                              | Strategia di divisione: <br>• `best` sceglie la miglior soglia <br>• `random` sceglie casualmente una soglia (utile per ridurre overfitting)                                                                                                              |
| `max_depth`                | `Integer` (es. `randint(2, 50)`)                                  | Profondità massima dell’albero. Controlla la complessità. <br>• Più alto → rischio overfitting <br>• Più basso → rischio underfitting                                                                                                                     |
| `min_samples_split`        | `Integer` o `Float` (es. `uniform(0.01, 0.3)` o `randint(2, 20)`) | Numero minimo di campioni richiesti per dividere un nodo. <br>Valori più alti rendono l’albero più semplice.                                                                                                                                              |
| `min_samples_leaf`         | `Integer` o `Float` (es. `uniform(0.01, 0.2)` o `randint(1, 10)`) | Numero minimo di campioni richiesti in una foglia. <br>Riduce l’overfitting e impone foglie più grandi.                                                                                                                                                   |
| `max_features`             | `["auto", "sqrt", "log2", None]` o `uniform(0.3, 1.0)`            | Numero di feature considerate per trovare la miglior divisione. <br>• `None`: usa tutte le feature <br>• `sqrt`, `log2`: randomizza il sottoinsieme delle feature                                                                                         |
| `max_leaf_nodes`           | `Integer` (es. `randint(10, 1000)` o `None`)                      | Numero massimo di foglie. <br>Limitandolo, si controlla la complessità del modello.                                                                                                                                                                       |
| `min_impurity_decrease`    | `Float` (es. `uniform(0.0, 0.02)`)                                | Una divisione viene fatta solo se riduce l’impurità di almeno questo valore.                                                                                                                                                                              |
| `ccp_alpha`                | `Float` (es. `uniform(0.0, 0.05)`)                                | Parametro di pruning (Cost Complexity Pruning). <br>Più alto → potatura più aggressiva → modello più semplice.                                                                                                                                            |
| `random_state`             | `Integer` o `None`                                                | Fissa la casualità per rendere i risultati riproducibili.                                                                                                                                                                                                 |
| `min_weight_fraction_leaf` | `Float` (es. `uniform(0.0, 0.5)`)                                 | Percentuale minima del peso totale dei campioni in una foglia. <br>Usato quando i dati hanno pesi.                                                                                                                                                        |


#### parametri miogliori per dati che tendono a RL

| Iperparametro           | Scelta/Range consigliato                                    | Perché                                                            |
| ----------------------- | ----------------------------------------------------------- | ----------------------------------------------------------------- |
| `criterion`             | `["squared_error", "absolute_error"]`                       | MSE per trend “puliti”; MAE se outlier.                           |
| `splitter`              | `["best"]` (eventuale `["best","random"]`)                  | Miglior soglia deterministica su singolo albero.                  |
| `max_depth`             | `randint(2, 8)`                                             | Basso → forma quasi lineare (pochi scalini).                      |
| `min_samples_split`     | `uniform(0.02, 0.28)` *(fraz.)* **oppure** `randint(2, 20)` | Più alto → meno partizioni. Preferisco frazione per scalabilità.  |
| `min_samples_leaf`      | `uniform(0.02, 0.15)` *(frazione del dataset)*              | Foglie grandi → funzione più liscia.                              |
| `max_features`          | `[None, 0.7, 0.85, 1.0, "sqrt"]`                            | Se poche feature utili, None/1.0; includo 0.7–0.85 per sicurezza. |
| `max_leaf_nodes`        | `randint(6, 40)`                                            | Limita la complessità; con trend lineare bastano poche foglie.    |
| `min_impurity_decrease` | `uniform(0.0, 0.002)`                                       | Evita split con guadagno trascurabile.                            |
| `ccp_alpha`             | `uniform(0.0, 0.02)`                                        | Potatura cost-complexity leggera–media.                           |
| `random_state`          | fisso (es. `42`)                                            | Riproducibilità.                                                  |


In [14]:
# ============================================
# SEZIONE: Ottimizzazione Iperparametri Decision Tree (RandomizedSearchCV)
# ============================================
# RandomizedSearchCV cerca i migliori iperparametri provando combinazioni casuali
# È più veloce di GridSearchCV quando lo spazio di ricerca è grande

import numpy as np
from scipy.stats import randint, uniform
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

# Creiamo un Decision Tree base con parametri iniziali
dt = DecisionTreeRegressor(max_depth=3, random_state=42)

# Definiamo i range di iperparametri da esplorare
# Questi range sono ottimizzati per dati con trend tendenzialmente lineare
param_dist = {
    # Criterio per misurare la qualità di una divisione
    "criterion": ["squared_error", "absolute_error", "friedman_mse"],
    
    # Profondità massima dell'albero (None = nessun limite)
    # Valori bassi = più regolarizzazione, meno overfitting
    "max_depth": [3, 4, 5, 10, None],
    
    # Numero minimo di campioni richiesti per essere una foglia
    # Valori più alti = più regolarizzazione
    "min_samples_leaf": [0.02, 0.03, 0.05, 1, 2, 5],
    
    # Numero minimo di campioni richiesti per dividere un nodo
    "min_samples_split": [0.05, 0.1, 0.15, 2, 5, 10, 20],
    
    # Numero massimo di foglie (None = nessun limite)
    "max_leaf_nodes": [8, 12, 16, 24, 32, None],
}

# Creiamo il RandomizedSearchCV
# IMPORTANTE: usiamo np.argmin perché vogliamo MINIMIZZARE il MAE
# (i punteggi sono negativi in sklearn, quindi il valore più alto è il migliore,
#  ma quando usiamo argmin cerchiamo direttamente il valore meno negativo = MAE più basso)
dt_rscv = RandomizedSearchCV(
    dt,
    param_distributions=param_dist,
    scoring=scoring,              # Dizionario con metriche: {'MAE', 'MSE', 'RMSE'}
    n_iter=1000,                  # Numero di combinazioni casuali da provare
    return_train_score=True,      # Restituisce anche i punteggi sul training set
    refit=lambda cvres: np.argmin(cvres['mean_test_MAE']),  # Seleziona il modello con MAE minimo
    cv=kf,                        # Strategia di cross-validazione (KFold)
    verbose=1,                    # Mostra progressi durante l'esecuzione
    n_jobs=-1,                    # Usa tutti i core disponibili
    random_state=42,              # Per risultati riproducibili
)

# Addestriamo il modello con tutti i parametri da provare
dt_rscv.fit(X_train, y_train)

# Otteniamo i migliori parametri trovati
best_params = dt_rscv.best_params_

# Creiamo una tabella con i migliori iperparametri
best_params_table = pd.DataFrame(
    list(best_params.items()), 
    columns=['Hyperparameter', 'Best Value']
)

# Mostriamo la tabella con i risultati
print("Best RandomizedSearchCV Parameters:\n")
print(best_params_table.to_string(index=False))


Fitting 10 folds for each of 1000 candidates, totalling 10000 fits
Best RandomizedSearchCV Parameters:

   Hyperparameter   Best Value
min_samples_split            5
 min_samples_leaf            2
   max_leaf_nodes           32
        max_depth            3
        criterion friedman_mse


In [15]:
# ============================================
# Risultati del Miglior Decision Tree
# ============================================
# Mostriamo le prestazioni del miglior modello trovato da RandomizedSearchCV

# Otteniamo l'indice del miglior modello
best_idx = dt_rscv.best_index_

# Mostriamo i punteggi sul training set
print('Train RMSE:', dt_rscv.cv_results_['mean_train_RMSE'][best_idx])
print('Train MAE:', dt_rscv.cv_results_['mean_train_MAE'][best_idx])

# Mostriamo i punteggi sul test set (cross-validation)
print('Test RMSE:', dt_rscv.cv_results_['mean_test_RMSE'][best_idx])
print('Test MAE:', dt_rscv.cv_results_['mean_test_MAE'][best_idx])


Train RMSE: 23644.22
Validation RMSE: 27647.42

Train MAE: 17843.83
Validation MAE: 20412.32


### Hyperparameters for the tree-based GridSearchCV

### Hyperparameters for the kNN RandomizedSearchCV

### Hyperparameters for the kNN GridSearchCV

### Hyperparameters for the GradientBoostingRegressor RandomizedSearchCV

In [30]:
# ============================================
# Ottimizzazione GradientBoostingRegressor (RandomizedSearchCV)
# ============================================
# GradientBoosting è un algoritmo potente che combina molti alberi decisionali "deboli"
# per creare un modello molto accurato. Ogni albero corregge gli errori del precedente.

import numpy as np
from scipy.stats import randint, uniform, loguniform
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# Creiamo il modello GradientBoosting base
gbr = GradientBoostingRegressor(random_state=42)

# Definiamo i range di iperparametri da esplorare
param_dist = {
    # Funzione di perdita da ottimizzare:
    # - squared_error: MSE standard
    # - huber: robusta agli outlier (compromesso tra MSE e MAE)
    # - absolute_error: MAE (più robusta ma più lenta)
    "loss": ["squared_error", "huber", "absolute_error"],
    
    # Learning rate (tasso di apprendimento):
    # - Valori bassi (0.01-0.1) = apprendimento lento ma accurato
    # - Valori alti = apprendimento veloce ma rischio di overfitting
    "learning_rate": loguniform(0.01, 0.3),
    
    # Numero di alberi:
    # - Più alberi = modello più complesso
    # - Troppi alberi = rischio di overfitting
    "n_estimators": randint(50, 500),
    
    # Profondità massima di ogni albero:
    # - 3-5: alberi semplici, meno overfitting
    # - 10+: alberi complessi, più accurati ma rischio overfitting
    "max_depth": randint(3, 15),
    
    # Numero minimo di campioni per dividere un nodo:
    "min_samples_split": randint(2, 20),
    
    # Numero minimo di campioni per essere una foglia:
    "min_samples_leaf": randint(1, 10),
    
    # Frazione di campioni usati per addestrare ogni albero:
    # - 1.0: usa tutti i dati
    # - <1.0: usa solo una frazione (stochastic gradient boosting, riduce overfitting)
    "subsample": uniform(0.6, 0.4),  # range: 0.6 to 1.0
    
    # Numero massimo di features da considerare per ogni split:
    "max_features": ["sqrt", "log2", None],
}

# Creiamo RandomizedSearchCV per trovare i migliori iperparametri
gbr_rscv = RandomizedSearchCV(
    gbr,
    param_distributions=param_dist,
    scoring=scoring,              # Usiamo le metriche definite prima (MSE, RMSE, MAE)
    n_iter=1000,                  # Proviamo 1000 combinazioni casuali
    return_train_score=True,      # Restituisce anche i punteggi sul training
    refit='MAE',                  # Usa MAE per selezionare il miglior modello
    cv=kf,                        # Usa KFold cross-validation
    verbose=1,                    # Mostra progressi
    n_jobs=-1,                    # Usa tutti i core CPU disponibili
    random_state=42,              # Per risultati riproducibili
)

# Addestriamo il modello provando tutte le combinazioni
gbr_rscv.fit(X_train, y_train)

# Otteniamo i migliori parametri
best_params = gbr_rscv.best_params_

# Creiamo una tabella con i migliori iperparametri
best_params_table = pd.DataFrame(
    list(best_params.items()), 
    columns=['Hyperparameter', 'Best Value']
)

# Mostriamo la tabella
print("Best GradientBoostingRegressor Parameters:\n")
print(best_params_table.to_string(index=False))


Fitting 10 folds for each of 150 candidates, totalling 1500 fits

 Best GradientBoostingRegressor Parameters:

       Hyperparameter    Best Value
                alpha       0.90759
            ccp_alpha      0.006995
            criterion squared_error
        learning_rate      0.062895
                 loss squared_error
            max_depth             3
         max_features          sqrt
min_impurity_decrease      0.001429
     min_samples_leaf      0.013696
    min_samples_split      0.091788
         n_estimators           890
     n_iter_no_change            14
            subsample      0.985059
                  tol       0.00047
  validation_fraction      0.169597

 Model Performance Summary:

Metric   Train  Validation  Difference  % Diff
  RMSE 8586.46    13250.83     4664.37   54.32
   MAE 6256.78     9777.30     3520.52   56.27


Il codice SOTTO serve per identificare e mostrare i 5 modelli PEGGIORI NO migliori in base alla media di MAE (Mean Absolute Error) durante la ricerca randomizzata

In [31]:
# ============================================
# Visualizzazione dei 5 Migliori Modelli (Prima Versione)
# ============================================
# Questa cella mostra i 5 modelli con MAE più basso dalla prima ricerca

# Ottieni gli indici dei 5 modelli con MAE più basso
# argsort ordina i valori e restituisce gli indici
# [:5] prende solo i primi 5
best_indices = np.argsort(gbr_rscv.cv_results_['mean_test_MAE'])[:5]

# Cicla attraverso i 5 migliori modelli e mostra i risultati
print("Top 5 modelli con MAE più basso:\n")
for rank, i in enumerate(best_indices, 1):
    mae = abs(gbr_rscv.cv_results_['mean_test_MAE'][i])
    print(f"Rank {rank}: MAE = {mae:.3f}")
    print("Parametri:", gbr_rscv.cv_results_['params'][i])
    print()


MAE: -33264.92
Hyperparams: {'alpha': np.float64(0.8558930465490906), 'ccp_alpha': np.float64(0.0001787393473341381), 'criterion': 'squared_error', 'learning_rate': np.float64(0.001293361253283509), 'loss': 'absolute_error', 'max_depth': 2, 'max_features': None, 'min_impurity_decrease': np.float64(0.0013592945653861396), 'min_samples_leaf': np.float64(0.029642849907855776), 'min_samples_split': np.float64(0.19099301311104055), 'n_estimators': 210, 'n_iter_no_change': 9, 'subsample': np.float64(0.6357644009249038), 'tol': np.float64(6.841661003247826e-05), 'validation_fraction': np.float64(0.18791183075621648)}

MAE: -32244.11
Hyperparams: {'alpha': np.float64(0.8794489411886416), 'ccp_alpha': np.float64(0.003460165676448396), 'criterion': 'squared_error', 'learning_rate': np.float64(0.0010573130043579609), 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'min_impurity_decrease': np.float64(0.0001757329982896667), 'min_samples_leaf': np.float64(0.04155642380882627), 'min_s

secondo round

In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ⚠️ Prerequisiti: gbr_rscv è già fit-tato; X_train, X_test, y_train, y_test sono definiti.

# 1) Seleziona i 5 migliori in base a MAE (convertito in positivo)
cv_mae_neg = gbr_rscv.cv_results_['mean_test_MAE']            # valori NEGATIVI (neg_mean_absolute_error)
cv_rmse_neg = gbr_rscv.cv_results_.get('mean_test_RMSE', None)  # opzionale
cv_mae = -cv_mae_neg                                           # MAE positivo
cv_rmse = -cv_rmse_neg if cv_rmse_neg is not None else None

best_indices = np.argsort(cv_mae)[:5]   # più piccolo MAE → migliore

# 2) Allena ciascun modello e valuta su train/test
round2_rows = []
for rank, i in enumerate(best_indices, start=1):
    params = gbr_rscv.cv_results_['params'][i]

    # Modello Round 2 con gli iperparametri del candidato
    model = GradientBoostingRegressor(random_state=42, **params)
    model.fit(X_train, y_train)

    # Predizioni
    y_pred_tr = model.predict(X_train)
    y_pred_te = model.predict(X_test)


    # Calcola MAE e RMSE per il training e la validation
    mae_tr = mean_absolute_error(y_train, y_pred_tr)
    mae_te = mean_absolute_error(y_test,  y_pred_te)
    rmse_tr = np.sqrt(mean_squared_error(y_train, y_pred_tr))
    rmse_te = np.sqrt(mean_squared_error(y_test,  y_pred_te))

    row = {
        "Rank (by CV-MAE)": rank,
        "CV MAE (mean)": float(cv_mae[i]),
        **({"CV RMSE (mean)": float(cv_rmse[i])} if cv_rmse is not None else {}),
        "Train MAE": mae_tr,
        "Test MAE":  mae_te,
        "Train RMSE": rmse_tr,
        "Test RMSE":  rmse_te,
        "Params": params
    }
    round2_rows.append(row)

# 3) Tabella finale ordinata per Test MAE
round2_df = pd.DataFrame(round2_rows)
round2_df = round2_df.sort_values(by="Test MAE", ascending=True).reset_index(drop=True)

print("\n🔁 Round 2 — Top-5 da CV per MAE (valutati su Train/Test):\n")
pd.set_option('display.max_colwidth', None)
print(round2_df.to_string(index=False))

# (Opzionale) Modello migliore su Test MAE 
best_round2_params = round2_df.iloc[0]["Params"]
best_round2_model = GradientBoostingRegressor(random_state=42, **best_round2_params).fit(X_train, y_train)



🔁 Round 2 — Top-5 da CV per MAE (valutati su Train/Test):

 Rank (by CV-MAE)  CV MAE (mean)  CV RMSE (mean)   Train MAE     Test MAE   Train RMSE    Test RMSE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Params
                1    9777.295692    13250.829828 5754.962601  9904.987588  7853.283707 14626.994572  {'alpha': 0.907589546707935, 'ccp_alpha': 0.0069951221076719385, 'criterion': 'squared_error', 'learning_rate': 0.06289502232749172, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'min_impurity_decrease': 0.0014291902083599043, 'min_samples_

Pipeline

In [33]:
import numpy as np
from scipy.stats import randint, uniform, loguniform
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# Se hai solo feature numeriche, puoi usare preprocessor="passthrough"
# Qui mostro un setup tipico con numeriche + categoriche
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_features),   # scaling leggero (GBR non richiede scaling, ma non fa danni)
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ],
    remainder="drop"
)

pipe = Pipeline([
    ("pre", preprocessor),
    ("gbr", GradientBoostingRegressor(random_state=42)),
])

param_dist = {
    "gbr__loss": ["squared_error", "huber"],
    "gbr__learning_rate": loguniform(1e-3, 3e-1),
    "gbr__n_estimators": randint(150, 1201),
    "gbr__subsample": uniform(0.6, 0.4),       # 0.6–1.0
    "gbr__max_depth": randint(2, 5),           # 2–4
    "gbr__min_samples_split": uniform(0.02, 0.18),
    "gbr__min_samples_leaf": uniform(0.01, 0.09),
    "gbr__max_features": [None, 1.0, 0.8, "sqrt"],
    "gbr__criterion": ["friedman_mse", "squared_error"],
    "gbr__min_impurity_decrease": uniform(0.0, 0.002),
    "gbr__ccp_alpha": uniform(0.0, 0.01),
    # early-stopping interno del GBR
    "gbr__validation_fraction": uniform(0.1, 0.1),  # 0.10–0.20
    "gbr__n_iter_no_change": randint(5, 16),
    "gbr__tol": loguniform(1e-5, 1e-3),
}

# Consiglio: usa scorer ufficiali negativi e lascia a sklearn scegliere il migliore con refit="RMSE"
scoring = {
    "RMSE": "neg_root_mean_squared_error",
    "MAE": "neg_mean_absolute_error",
}

gbr_rscv = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=80,            # compatto ma efficace
    cv=kf,
    scoring=scoring,
    refit="RMSE",         # migliore per RMSE (valori più "alti" perché negativi → migliore)
    n_jobs=-1,
    verbose=1,
    random_state=42,
    return_train_score=True,
)

gbr_rscv.fit(X_train, y_train)
print("Best params:", gbr_rscv.best_params_)


Fitting 10 folds for each of 80 candidates, totalling 800 fits
Best params: {'gbr__ccp_alpha': np.float64(0.0029348817471803812), 'gbr__criterion': 'friedman_mse', 'gbr__learning_rate': np.float64(0.10471209213501693), 'gbr__loss': 'squared_error', 'gbr__max_depth': 4, 'gbr__max_features': 0.8, 'gbr__min_impurity_decrease': np.float64(0.0015425406933718916), 'gbr__min_samples_leaf': np.float64(0.016664018656068133), 'gbr__min_samples_split': np.float64(0.08452383113796907), 'gbr__n_estimators': 190, 'gbr__n_iter_no_change': 11, 'gbr__subsample': np.float64(0.9400154311159197), 'gbr__tol': np.float64(7.923213310872074e-05), 'gbr__validation_fraction': np.float64(0.10954101164904113)}


In [34]:
# ============================================
# Prestazioni del Miglior Modello
# ============================================
# Mostriamo le metriche del miglior modello GradientBoosting trovato

print(" Best GradientBoostingRegressor Performance:\n")

# Mostriamo i punteggi sul training set (valori negativi convertiti in positivi)
print("Train RMSE:", (-gbr_rscv.cv_results_['mean_train_RMSE'][gbr_rscv.best_index_]))
print("Train MAE:", (-gbr_rscv.cv_results_['mean_train_MAE'][gbr_rscv.best_index_]))

# Mostriamo i punteggi sul test set (cross-validation)
print("Test RMSE:", (-gbr_rscv.cv_results_['mean_test_RMSE'][gbr_rscv.best_index_]))
print("Test MAE:", (-gbr_rscv.cv_results_['mean_test_MAE'][gbr_rscv.best_index_]))


 Best GradientBoostingRegressor Performance:

Train RMSE: 7802.47
Validation RMSE: 13579.44

Train MAE: 5743.85
Validation MAE: 10014.76


# Dopo correzione prof 

Ora sono i 5 positivi

In [24]:
# ============================================
# Visualizzazione dei 5 Migliori Risultati
# ============================================
# Dopo aver eseguito RandomizedSearchCV, vogliamo vedere i 5 migliori modelli
# trovati in base al Mean Absolute Error (MAE)

# IMPORTANTE: I punteggi MAE in cv_results_ sono già NEGATIVI
# Quindi per trovare i 5 MAE più bassi (migliori), cerchiamo i valori
# MENO negativi (più vicini a zero), che sono i valori più ALTI
# argsort ordina dal più piccolo al più grande, quindi prendiamo gli ultimi 5

# Ottieni gli indici dei 5 migliori modelli (MAE più basso = valore meno negativo)
# Non usiamo il segno meno perché i valori sono già negativi
best_indices = np.argsort(gbr_rscv.cv_results_['mean_test_MAE'])[:5]

# Mostra i 5 migliori risultati con MAE positivo (più facile da leggere)
print("Top 5 migliori combinazioni per MAE:\n")
for i in best_indices:
    # Prendiamo il valore assoluto per mostrare MAE come numero positivo
    mae = abs(gbr_rscv.cv_results_['mean_test_MAE'][i])
    print(f"Rank {i}: MAE = {mae:.3f}")
    print("Hyperparams:", gbr_rscv.cv_results_['params'][i])
    print()


Top 5 migliori combinazioni per MAE:

Rank 22: MAE = 9777.296
Hyperparams: {'alpha': np.float64(0.907589546707935), 'ccp_alpha': np.float64(0.0069951221076719385), 'criterion': 'squared_error', 'learning_rate': np.float64(0.06289502232749172), 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'min_impurity_decrease': np.float64(0.0014291902083599043), 'min_samples_leaf': np.float64(0.01369607650910882), 'min_samples_split': np.float64(0.09178776226006302), 'n_estimators': 890, 'n_iter_no_change': 14, 'subsample': np.float64(0.98505936587117), 'tol': np.float64(0.00046985109250894774), 'validation_fraction': np.float64(0.1695974206093698)}

Rank 20: MAE = 9941.017
Hyperparams: {'alpha': np.float64(0.8507258807883581), 'ccp_alpha': np.float64(0.006278944149486361), 'criterion': 'squared_error', 'learning_rate': np.float64(0.04074173013132515), 'loss': 'huber', 'max_depth': 3, 'max_features': 'sqrt', 'min_impurity_decrease': np.float64(0.0017732342979013198), 'min_samples_l

secondo round

In [25]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ⚠️ Prerequisiti: gbr_rscv è già fit-tato; X_train, X_test, y_train, y_test, kf definiti.

# 1) Prendi i 5 migliori in base a MAE di CV (positivo)
cv_mae_neg = gbr_rscv.cv_results_['mean_test_MAE']              # negativo
cv_rmse_neg = gbr_rscv.cv_results_.get('mean_test_RMSE', None)  # negativo (se presente)

cv_mae = -cv_mae_neg
cv_rmse = -cv_rmse_neg if cv_rmse_neg is not None else None

best_indices = np.argsort(cv_mae)[:5]  # MAE più piccolo → migliore

# 2) Refit dei 5 migliori e valutazione su Train/Test
rows = []
for rank, i in enumerate(best_indices, start=1):
    params = gbr_rscv.cv_results_['params'][i]

    model = GradientBoostingRegressor(random_state=42, **params)
    model.fit(X_train, y_train)

    y_pred_tr = model.predict(X_train)
    y_pred_te = model.predict(X_test)

    mae_tr = mean_absolute_error(y_train, y_pred_tr)
    mae_te = mean_absolute_error(y_test,  y_pred_te)
    rmse_tr = np.sqrt(mean_squared_error(y_train, y_pred_tr))
    rmse_te = np.sqrt(mean_squared_error(y_test,  y_pred_te))

    row = {
        "Rank by CV-MAE": rank,
        "CV MAE (mean)": float(cv_mae[i]),
        **({"CV RMSE (mean)": float(cv_rmse[i])} if cv_rmse is not None else {}),
        "Train MAE": mae_tr,
        "Test MAE":  mae_te,
        "Train RMSE": rmse_tr,
        "Test RMSE":  rmse_te,
        "Params": params
    }
    rows.append(row)

# 3) Tabella finale ordinata per Test MAE (migliore in alto)
df_round2 = pd.DataFrame(rows).sort_values(by="Test MAE", ascending=True).reset_index(drop=True)

print("\n🔁 Round 2 — Top-5 (refit con stessi iperparametri, metriche positive):\n")
pd.set_option('display.max_colwidth', None)
print(df_round2.to_string(index=False))

# (Opzionale) prendi il migliore del Round 2 e tienilo pronto
best_params_r2 = df_round2.iloc[0]["Params"]
best_model_r2 = GradientBoostingRegressor(random_state=42, **best_params_r2).fit(X_train, y_train)



🔁 Round 2 — Top-5 (refit con stessi iperparametri, metriche positive):

 Rank by CV-MAE  CV MAE (mean)  CV RMSE (mean)   Train MAE     Test MAE   Train RMSE    Test RMSE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Params
              1    9777.295692    13250.829828 5754.962601  9904.987588  7853.283707 14626.994572  {'alpha': 0.907589546707935, 'ccp_alpha': 0.0069951221076719385, 'criterion': 'squared_error', 'learning_rate': 0.06289502232749172, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'min_impurity_decrease': 0.0014291902083599043, 'min

pipeline

In [27]:
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform, loguniform
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# Se hai solo feature numeriche, puoi usare preprocessor="passthrough"
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_features),  # ok con OHE (sparse)
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ],
    remainder="drop"
)

pipe = Pipeline([
    ("pre", preprocessor),
    ("gbr", GradientBoostingRegressor(random_state=42)),
])

param_dist = {
    "gbr__loss": ["squared_error", "huber"],
    "gbr__learning_rate": loguniform(1e-3, 3e-1),
    "gbr__n_estimators": randint(150, 1201),
    "gbr__subsample": uniform(0.6, 0.4),       # 0.6–1.0
    "gbr__max_depth": randint(2, 5),           # 2–4
    "gbr__min_samples_split": uniform(0.02, 0.18),
    "gbr__min_samples_leaf": uniform(0.01, 0.09),
    "gbr__max_features": [None, 1.0, 0.8, "sqrt"],
    "gbr__criterion": ["friedman_mse", "squared_error"],
    "gbr__min_impurity_decrease": uniform(0.0, 0.002),
    "gbr__ccp_alpha": uniform(0.0, 0.01),
    # early-stopping interno del GBR
    "gbr__validation_fraction": uniform(0.1, 0.1),  # 0.10–0.20
    "gbr__n_iter_no_change": randint(5, 16),
    "gbr__tol": loguniform(1e-5, 1e-3),
}

# Usiamo gli scorer "neg_*" standard, ma poi li mostriamo in positivo
scoring = {
    "RMSE": "neg_root_mean_squared_error",
    "MAE": "neg_mean_absolute_error",
}

gbr_rscv = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=80,
    cv=kf,                 # il tuo KFold
    scoring=scoring,
    refit="RMSE",
    n_jobs=-1,
    verbose=1,
    random_state=42,
    return_train_score=True,
)

gbr_rscv.fit(X_train, y_train)

# ------------------------------
# Report in POSITIVO (senza cambiare la logica del search)
# ------------------------------
print("\nBest params:", gbr_rscv.best_params_)

idx = gbr_rscv.best_index_
train_rmse = abs(gbr_rscv.cv_results_['mean_train_RMSE'][idx])
val_rmse   = abs(gbr_rscv.cv_results_['mean_test_RMSE'][idx])
train_mae  = abs(gbr_rscv.cv_results_['mean_train_MAE'][idx])
val_mae    = abs(gbr_rscv.cv_results_['mean_test_MAE'][idx])

metrics_table = pd.DataFrame({
    'Metric': ['RMSE', 'MAE'],
    'Train': [round(train_rmse, 4), round(train_mae, 4)],
    'Validation': [round(val_rmse, 4), round(val_mae, 4)],
})
metrics_table['Difference'] = (metrics_table['Validation'] - metrics_table['Train']).round(4)
metrics_table['% Diff'] = ((metrics_table['Difference'] / metrics_table['Train']) * 100).round(2)

print("\nModel Performance Summary (positive values):\n")
print(metrics_table.to_string(index=False))

# ------------------------------
# (Opzionale) Top-5 per MAE (positivi) dalla CV
# ------------------------------
cv_mae_pos = -gbr_rscv.cv_results_['mean_test_MAE']
top5_idx = np.argsort(cv_mae_pos)[:5]  # MAE più piccolo → migliore

print("\nTop-5 by CV MAE (positive):\n")
for i in top5_idx:
    mae = cv_mae_pos[i]
    rmse = -gbr_rscv.cv_results_['mean_test_RMSE'][i]
    print(f"MAE={mae:.4f} | RMSE={rmse:.4f} | params={gbr_rscv.cv_results_['params'][i]}")


Fitting 10 folds for each of 80 candidates, totalling 800 fits

Best params: {'gbr__ccp_alpha': np.float64(0.0029348817471803812), 'gbr__criterion': 'friedman_mse', 'gbr__learning_rate': np.float64(0.10471209213501693), 'gbr__loss': 'squared_error', 'gbr__max_depth': 4, 'gbr__max_features': 0.8, 'gbr__min_impurity_decrease': np.float64(0.0015425406933718916), 'gbr__min_samples_leaf': np.float64(0.016664018656068133), 'gbr__min_samples_split': np.float64(0.08452383113796907), 'gbr__n_estimators': 190, 'gbr__n_iter_no_change': 11, 'gbr__subsample': np.float64(0.9400154311159197), 'gbr__tol': np.float64(7.923213310872074e-05), 'gbr__validation_fraction': np.float64(0.10954101164904113)}

Model Performance Summary (positive values):

Metric     Train  Validation  Difference  % Diff
  RMSE 7802.4680  13579.4435   5776.9755   74.04
   MAE 5743.8475  10014.7630   4270.9155   74.36

Top-5 by CV MAE (positive):

MAE=10014.7630 | RMSE=13579.4435 | params={'gbr__ccp_alpha': np.float64(0.002934881

In [28]:
# ============================================
# Prestazioni del Modello (Valori Positivi)
# ============================================
# Visualizziamo le stesse metriche ma con valori positivi per maggiore chiarezza

print("\n--- Model Performance (positive values) ---\n")

# Convertiamo i valori negativi in positivi per renderli più leggibili
print('Train RMSE:', (-gbr_rscv.cv_results_['mean_train_RMSE'][gbr_rscv.best_index_]))
print('Train MAE:', (-gbr_rscv.cv_results_['mean_train_MAE'][gbr_rscv.best_index_]))
print('Test RMSE:', (-gbr_rscv.cv_results_['mean_test_RMSE'][gbr_rscv.best_index_]))
print('Test MAE:', (-gbr_rscv.cv_results_['mean_test_MAE'][gbr_rscv.best_index_]))



--- Model Performance (positive values) ---

Train RMSE: 7802.47
Validation RMSE: 13579.44

Train MAE: 5743.85
Validation MAE: 10014.76


In [35]:
# Visualizziamo il DataFrame di leaderboard prima della pulizia
# Questo mostra ancora tutte le colonne originali
leaderboard_test_df


,Split,Id,LotArea,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,EnclosedPorch,PoolArea,MiscFeature,SalePrice,mszoning_num,HasMiscFeature,HouseStyle_num,st_grvl_pave,ut_allpub_nosewa,land_slope_ord,roof_style_num,foundation_num,heating_num,electrical_num,central_air_num,garage_num,garage_finish_num,paveddrive_num
794,leaderboard,795,9828,1995.0,1995,1128,1142,878,2020,0,2,3,1,8,1,1995.0,2,466,155,0,0,0,NaN,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,2,1.0
795,leaderboard,796,10206,1952.0,1952,0,944,0,944,0,1,2,1,4,0,1956.0,2,528,0,0,0,0,NaN,3.0,0,1.0,1,1,0,0.0,0.0,2.0,1.0,0,2.0,1,1.0
796,leaderboard,797,11796,2004.0,2005,847,847,1112,1959,0,2,4,1,8,1,2004.0,2,434,48,0,0,0,NaN,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,4.0,3,1.0
797,leaderboard,798,6000,1926.0,2004,884,904,0,904,0,1,2,1,4,0,1926.0,1,180,0,105,0,0,NaN,2.0,0,1.5,1,1,0,2.0,3.0,3.0,3.0,1,2.0,1,1.0
798,leaderboard,799,11367,2002.0,2002,1065,1091,898,1989,1,2,3,1,7,1,2002.0,2,586,60,0,0,0,NaN,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,leaderboard,988,8100,1948.0,1973,1559,1559,0,1559,1,1,2,1,5,0,1948.0,2,812,116,230,0,0,NaN,2.0,0,1.0,1,1,0,2.0,1.0,3.0,3.0,1,2.0,1,1.0
988,leaderboard,989,2645,1999.0,2000,970,983,756,1739,1,2,3,1,7,0,1999.0,2,480,0,0,0,0,NaN,3.2,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
989,leaderboard,990,7060,1997.0,1998,1344,1344,0,1344,2,2,2,2,8,0,1997.0,4,784,0,0,0,0,NaN,2.0,0,3.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
990,leaderboard,991,6000,1939.0,1950,780,825,587,1412,0,1,4,1,6,1,1939.0,1,280,0,0,0,0,NaN,2.0,0,2.0,1,1,0,2.0,2.0,3.0,1.0,1,2.0,1,1.0


In [36]:
# ============================================
# Preparazione Dati per Predizione Finale
# ============================================
# Prima di fare le predizioni sul set di leaderboard, dobbiamo rimuovere
# le colonne che non sono features (caratteristiche) utilizzabili dal modello

# Rimuoviamo le colonne non necessarie:
# - 'Split': indica solo se il dato era training o test
# - 'Id': identificatore univoco, non utile per la predizione
# - 'SalePrice': il target (che per il leaderboard non conosciamo)
# 
# NOTA: quando usiamo 'columns=', non serve specificare 'axis=1'
# perché è già implicito che stiamo eliminando colonne
leaderboard_test_df = leaderboard_test_df.drop(columns=['Split', 'Id', 'SalePrice'])

# Visualizziamo il DataFrame risultante
leaderboard_test_df


,LotArea,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,EnclosedPorch,PoolArea,MiscFeature,mszoning_num,HasMiscFeature,HouseStyle_num,st_grvl_pave,ut_allpub_nosewa,land_slope_ord,roof_style_num,foundation_num,heating_num,electrical_num,central_air_num,garage_num,garage_finish_num,paveddrive_num
794,9828,1995.0,1995,1128,1142,878,2020,0,2,3,1,8,1,1995.0,2,466,155,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,2,1.0
795,10206,1952.0,1952,0,944,0,944,0,1,2,1,4,0,1956.0,2,528,0,0,0,0,3.0,0,1.0,1,1,0,0.0,0.0,2.0,1.0,0,2.0,1,1.0
796,11796,2004.0,2005,847,847,1112,1959,0,2,4,1,8,1,2004.0,2,434,48,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,4.0,3,1.0
797,6000,1926.0,2004,884,904,0,904,0,1,2,1,4,0,1926.0,1,180,0,105,0,0,2.0,0,1.5,1,1,0,2.0,3.0,3.0,3.0,1,2.0,1,1.0
798,11367,2002.0,2002,1065,1091,898,1989,1,2,3,1,7,1,2002.0,2,586,60,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,8100,1948.0,1973,1559,1559,0,1559,1,1,2,1,5,0,1948.0,2,812,116,230,0,0,2.0,0,1.0,1,1,0,2.0,1.0,3.0,3.0,1,2.0,1,1.0
988,2645,1999.0,2000,970,983,756,1739,1,2,3,1,7,0,1999.0,2,480,0,0,0,0,3.2,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
989,7060,1997.0,1998,1344,1344,0,1344,2,2,2,2,8,0,1997.0,4,784,0,0,0,0,2.0,0,3.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
990,6000,1939.0,1950,780,825,587,1412,0,1,4,1,6,1,1939.0,1,280,0,0,0,0,2.0,0,2.0,1,1,0,2.0,2.0,3.0,1.0,1,2.0,1,1.0


In [44]:
# Visualizziamo il miglior modello (estimator) trovato da RandomizedSearchCV
# Questo mostra i parametri finali del modello migliore
gbr_rscv.best_estimator_


,steps,"[('pre', ...), ('gbr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [45]:
# ============================================
# Predizione Finale sul Set di Leaderboard
# ============================================
# Ora che abbiamo trovato il miglior modello tramite RandomizedSearchCV,
# lo usiamo per fare le predizioni sul set di leaderboard

# Usiamo il miglior estimatore (modello) trovato da RandomizedSearchCV
# best_estimator_ contiene il modello già addestrato con i migliori iperparametri
# predict() fa le predizioni sulle nuove case (leaderboard_test_df)
predizione = gbr_rscv.best_estimator_.predict(leaderboard_test_df)


In [46]:
# Visualizziamo l'array con tutte le predizioni
# Questo ci mostra i prezzi predetti per ogni casa nel set di leaderboard
predizione


array([234821.11842902,  92538.96105623, 210214.47875379, 117654.02481564,
       235782.14163382,  90076.20497616, 140203.01301711, 277162.14688329,
       259398.69325475, 166294.46093385, 108191.68601022,  81064.55866393,
       242215.90059119,  90231.64830293, 193122.4509287 , 114130.32375927,
       182922.98342022, 218276.95391912, 222734.79135147, 227811.75262155,
       143793.37535016, 121673.51223353, 179141.32909027, 171826.58731839,
       174057.87851537, 131318.28635479, 354225.90540643, 270274.95121662,
       189460.84326927, 141599.67951837, 255068.13352233, 214650.706469  ,
       134805.59368641, 137039.61583185, 153051.08622606, 218727.23833586,
       343917.10103204, 180289.51698825, 140739.32942727, 219871.50893132,
       268313.51957283, 315860.436442  , 111081.2428244 , 119025.74401404,
       140129.84689819, 167828.30037442, 192267.77594405, 150254.42101823,
       158340.25497288, 243686.11536019, 128079.24171957, 203850.77527543,
       207183.68964812, 1

In [47]:
# ============================================
# Anteprima delle Prime Predizioni
# ============================================
# Visualizziamo le prime 10 predizioni per verificare che siano sensate

# Mostriamo i primi 10 valori predetti
# Questo ci aiuta a controllare che i prezzi siano realistici
print("Prime 10 predizioni:")
print(predizione[:10])


[234821.11842902  92538.96105623 210214.47875379 117654.02481564
 235782.14163382  90076.20497616 140203.01301711 277162.14688329
 259398.69325475 166294.46093385]


In [52]:
# Visualizziamo il DataFrame di leaderboard dopo la pulizia
# Ora contiene solo le features utilizzabili per la predizione
leaderboard_test_df


,LotArea,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,EnclosedPorch,PoolArea,MiscFeature,mszoning_num,HasMiscFeature,HouseStyle_num,st_grvl_pave,ut_allpub_nosewa,land_slope_ord,roof_style_num,foundation_num,heating_num,electrical_num,central_air_num,garage_num,garage_finish_num,paveddrive_num
794,9828,1995.0,1995,1128,1142,878,2020,0,2,3,1,8,1,1995.0,2,466,155,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,2,1.0
795,10206,1952.0,1952,0,944,0,944,0,1,2,1,4,0,1956.0,2,528,0,0,0,0,3.0,0,1.0,1,1,0,0.0,0.0,2.0,1.0,0,2.0,1,1.0
796,11796,2004.0,2005,847,847,1112,1959,0,2,4,1,8,1,2004.0,2,434,48,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,4.0,3,1.0
797,6000,1926.0,2004,884,904,0,904,0,1,2,1,4,0,1926.0,1,180,0,105,0,0,2.0,0,1.5,1,1,0,2.0,3.0,3.0,3.0,1,2.0,1,1.0
798,11367,2002.0,2002,1065,1091,898,1989,1,2,3,1,7,1,2002.0,2,586,60,0,0,0,3.0,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,8100,1948.0,1973,1559,1559,0,1559,1,1,2,1,5,0,1948.0,2,812,116,230,0,0,2.0,0,1.0,1,1,0,2.0,1.0,3.0,3.0,1,2.0,1,1.0
988,2645,1999.0,2000,970,983,756,1739,1,2,3,1,7,0,1999.0,2,480,0,0,0,0,3.2,0,5.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
989,7060,1997.0,1998,1344,1344,0,1344,2,2,2,2,8,0,1997.0,4,784,0,0,0,0,2.0,0,3.0,1,1,0,2.0,3.0,3.0,3.0,1,3.0,3,1.0
990,6000,1939.0,1950,780,825,587,1412,0,1,4,1,6,1,1939.0,1,280,0,0,0,0,2.0,0,2.0,1,1,0,2.0,2.0,3.0,1.0,1,2.0,1,1.0


In [55]:
# ============================================
# Creazione del File di Submission
# ============================================
# Creiamo un DataFrame con gli ID e le predizioni per la submission finale

# IMPORTANTE: range(795, 993) genera gli ID da 795 a 992
# Questo range è specifico per questo dataset e NON deve essere modificato
# Creiamo un DataFrame con:
# - 'id': gli identificatori delle case (da 795 a 992)
# - 'prediction': i prezzi predetti dal nostro modello
last_pred = pd.DataFrame({
    'id': range(795, 993), 
    'prediction': predizione
})

# Mostriamo le prime righe del DataFrame di submission
last_pred.head()


,id,prediction
0,795,234821.118429
1,796,92538.961056
2,797,210214.478754
3,798,117654.024816
4,799,235782.141634


In [56]:
# ============================================
# Salvataggio del File di Submission
# ============================================
# Salviamo le predizioni in un file CSV per la submission finale

# index=False: non salvare l'indice del DataFrame (solo id e prediction)
# Questo crea il file 'submission.csv' nella cartella corrente
last_pred.to_csv('submission.csv', index=False)

print("✓ File 'submission.csv' creato con successo!")
print(f"  Contiene {len(last_pred)} predizioni")
